<a href="https://colab.research.google.com/github/neverix/Pulkovo-Hack-Team-2020/blob/master/Schedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np

In [2]:
!git -C task pull || git clone https://github.com/practicingfutures/Pulkovo.Hack_task task
!cp -rf task/data .
!ls

Already up to date.
app3  data  gdrive  out.xlsx  sample_data  task


In [3]:
drop_cols = ["Начальная подготовка", "Начальная подготовка СД", "СПАСОП", "ПОЗ ВС", "Типы ВС", "ПТМ", "ЭБ", "Рабочий люльки"]


def read_1(filename="data/Приложение №1.xlsx"):
  df = pd.read_excel(filename)
  # переворачиваем
  df = df.T
  # выбираем категории (строка 5)
  cats = df.iloc[4]
  # выбираем названия (строка 7)
  df.columns = list(df.iloc[6])
  df = df.iloc[7:]
  # выбираем дни недели (столбец 2)
  df.index = df.iloc[:, 1]
  df = df.iloc[:, 2:]
  cats = cats[2:]
  # распространяем категории
  cats = cats[df.columns.notnull()]
  cats = cats.fillna(method="ffill")
  # убираем строки и столбцы с NaN
  df = df.loc[df.index.notnull(), df.columns.notnull()]
  # убираем "невидимые" столбцы
  df.drop(drop_cols, axis=1, inplace=True)
  
  for i in df.index:
    for k, j in enumerate(df.columns):
      x = df.loc[i, j]
      # делим на события
      x = x.split("\n\n") if isinstance(x, str) else []
      # делим неделю на даты
      start, end = [f for f in i.split('\n') if f][-1].split('-')
      # добавляем неделю и название
      x = [(y, start, end, cats.iloc[k], j) for y in x]
      # сохраняем результат
      df.loc[i, j] = x
  # уплощаем
  flat = list(df.values.flat)
  # совмещаем все клетки
  flat = sum(flat, [])

  # снова преобразуем
  for i, (text, start, end, cat, name) in enumerate(flat):
    texts = []
    aud = ''
    for d in text.split('\n'):
      if "ауд" in d:
        aud = d.split("ауд.")[-1]
      elif "ИАС" in d:
        aud = d.split("ИАС")[-1]
      elif '.' in d:
        if '-' in d:
          start, end = d.split('-')
        elif '–' in d:
          start, end = d.split('–')
        else:
          start = d
      else:
        texts.append(d)
    text = ' '.join(texts)
    flat[i] = text, aud, start, end, cat, name
  # превращаем обратно в df
  df = DataFrame(flat, columns=["text", "aud", "start", "end", "cat", "name"])
  # обрезаем пробелы
  df = df.apply(lambda x: x.str.strip())
  # объединяем группы, занимающие несколько рядов
  df.drop_duplicates(inplace=True)
  return df

example_calendar = read_1()
example_calendar

,text,aud,start,end,cat,name
0,Программа повышения квалификации,107,13.01,17.01,Обучение авиационного персонала,Досмотр
1,Специальная профессиональная подготовка,105,13.01,24.01,Обучение авиационного персонала,Перронный контроль
2,Повышение квалификации,414Б,13.01,14.01,Обучение неавиационного персонала,Опасные грузы. 10 категория
3,Повышение квалификации,414Б,15.01,16.01,Обучение неавиационного персонала,Опасные грузы. 10 категория
4,Повышение квалификации,422,13.01,14.01,Обучение неавиационного персонала,Опасные грузы. 9 категория
...,...,...,...,...,...,...
194,Повышение квалификации,422,14.12,15.12,Обучение неавиационного персонала,Опасные грузы. 9 категория
195,Повышение квалификации,416,15.12,16.12,Обучение неавиационного персонала,Опасные грузы. 8 категория
197,Базовый курс,414Б,21.12,22.12,Обучение неавиационного персонала,Опасные грузы. 9 категория
198,Базовый курс,108,23.12,25.12,Обучение неавиационного персонала,DCS Астра


In [4]:
def read_2():
  studies = pd.read_excel("data/Приложение №2.xlsx") 
  classes = pd.read_excel("data/Приложение №2.xlsx",1) 
  teachers = pd.read_excel("data/Приложение №2.xlsx",2)
  
  studies = studies.loc[:, "Учебная программа":]
  for i in studies.columns:
    if i!='Особенности проведения программы':
      studies[i] = studies[i].map(lambda x: 0 if x=='-' or  not pd.notnull(x) else x)
    else:
      studies[i] = studies[i].map(lambda x: 'нет' if not pd.notnull(x) else x)
  studies['Без парт'] = studies['Особенности проведения программы'].map(lambda x: 'без' in x.split())
  studies['Доска'] = studies['Особенности проведения программы'].map(lambda x: 'доска' in x.split())
  studies['Приоритет теория'] = studies['Особенности проведения программы'].map(lambda x: x.split()[-1].split(',') if 'аудитория' in x.split() and 'практические' not in x.split()  else [x.split()[-3], x.split()[-1]] if 'или' in x.split() else 0 )
  studies['Приоритет практика'] = studies['Особенности проведения программы'].map(lambda x: x.split()[-1].split(',') if 'практические' in x.split() else np.nan)
  
  classes["Аудитория"] = classes["Аудитрия"]
  classes["Без парт"] = classes["Конфигурация аудитории"].map(lambda x: "без" in x.split())
  classes['Доска'] = classes["Конфигурация аудитории"].map(lambda x: 'доска' in x.split())

  return studies, classes, teachers
studies, classes, teachers = read_2() 

In [5]:
studies

,Учебная программа,Особенности проведения программы,"Количество часов, ак.","Занятия в классе(теоретические и практические занятия), ак. часов","Практич. занятия на производстве, ак. часов","Обучение в СДО, часов",Количество дней очно,Количество групп в план-графике в 2020,Количество сотрудников ВВСС к обучению в 2020,"Количество внешних слушателей, которое может быть обучено в 2020 на базе АУЦ",Количество действующих преподавателей,Количество аудиторий для проведения,Без парт,Доска,Приоритет теория,Приоритет практика
0,"Программа повышения квалификации ""Центровка и...",требуется интерактивная доска,80.0,80,0,0,10,1.0,20.0,4,2,6,False,True,0,NaN
1,"Программа повышения квалификации ""Центровка и ...",требуется интерактивная доска,64.0,64,0,0,8,1.0,17.0,7,2,6,False,True,0,NaN
2,Программа повышения квалификации государственн...,"желательно все дни в одной и той же аудитории,...",40.0,40,0,0,5,0.0,0.0,0,4,7,False,False,0,NaN
3,Программа повышения квалификации руководящего...,желательно все дни в одной и той же аудитории,40.0,40,0,0,5,6.0,44.0,100,4,7,False,False,0,NaN
4,Программа повышения квалификации «Базовые комп...,необходима аудитория без парт: 414б-1 или 416аб,24.0,24,0,0,3,1.0,5.0,19,1,2,True,False,[416аб],NaN
5,Программа повышения квалификации «Подготовка п...,необходима аудитория без парт: 414б-1 или 416аб,16.0,16,0,0,2,1.0,10.0,14,1,2,True,False,[416аб],NaN
6,Программа повышения квалификации «Перевозка оп...,нет,24.0,24,0,0,3,1.0,20.0,4,2,7,False,False,0,NaN
7,Программа повышения квалификации «Перевозка оп...,нет,16.0,16,0,0,2,5.0,88.0,32,2,7,False,False,0,NaN
8,Программа повышения квалификации «Перевозка оп...,нет,16.0,16,0,0,2,4.0,50.0,46,2,7,False,False,0,NaN
9,Программа повышения квалификации «Перевозка оп...,нет,16.0,8,0,8,2,6.0,124.0,20,2,7,False,False,0,NaN


In [6]:
classes

,Аудитрия,Количество мест,Вид занятий,Конфигурация аудитории,Преимущество у дисциплины,Подходит для дисциплин,Аудитория,Без парт,Доска
0,105,18,теоретические,с партами,Авиационная безопасность,кроме Подготовка преподавателей АУЦ,105,False,False
1,106,20,теоретические,с партами,Авиационная безопасность,кроме Подготовка преподавателей АУЦ,106,False,False
2,107,24,практические/теоретические,с партами,Авиационная безопасность,кроме Подготовка преподавателей АУЦ,107,False,False
3,414б,24,теоретические,"с партами, интерактивная доска",Центровка и контроль загрузки,кроме Подготовка преподавателей АУЦ,414б,False,True
4,414б-1,32,теоретические/практические,"стулья без парт, для тренинговых форматов",нет,все дисциплины,414б-1,True,False
5,"416а,б",18,практические/теоретические,"стулья без парт, для тренинговых форматов",Авиационная безопасность,все дисциплины,"416а,б",True,False
6,422,18,теоретические,с партами,нет,кроме Подготовка преподавателей АУЦ,422,False,False
7,3091,20,практические,"с партами,\n муляжи для Авиационной безопасности",нет,"Авиационная безопасность, \nтолько для практич...",3091,False,False
8,ГАСС,20,практические/теоретические,с партами,нет,Аварийно-спасательное обеспечение полетов,ГАСС,False,False
9,"ИАС, 202",20,теоретические,с партами,Организация наземного обслуживания,Организация наземного обслуживания;\nЦентровка...,"ИАС, 202",False,False


In [7]:
teachers

,Табельный номер преподавателя,Преподаватель,Дисциплина,Учебные программы,Приоритет при распределении,Может проводить занятия по темам,График работы,График сменности
0,20435,Лебедев М.В.,Центровка и контроль загрузки,1;2,2,все темы,пятидневный,нет
1,33688,Красненко А.Г.,Центровка и контроль загрузки,1;2,1,все темы,пятидневный,нет
2,32909,Морозов И.О.,Управление безопасностью полетов,3;4,1,все темы,пятидневный,нет
3,40825,Корнеев Р.Л.,Управление безопасностью полетов,3;4,2,все темы,пятидневный,нет
4,42927,Чернов Д.А.,Управление безопасностью полетов,3;4,3,все темы,пятидневный,нет
5,12426,Автеньев Д.Г.,Опасные грузы,7;8;11;12,2,все темы,сменный,3 смена
6,31787,Шабалин К.Н.,Опасные грузы,7;8;11;12,при распределении на программы 7 и 8 - приорит...,все темы,пятидневный,нет
7,35684,Пачко Г.М.,Опасные грузы,11;12,при распределении на программы 11 и 12- приори...,все темы,пятидневный,нет
8,37590,Гладковская А.Ю.,Опасные грузы; Организация пассажирских перевозок,11;12;9;10;13;14,при распределении на программы 11;12 приоритет 2,все темы,пятидневный,нет
9,6841,Попова И.О.,Опасные грузы; Организация пассажирских перевозок,11;12;9;10;13;14,при распределении на программы 11;12 приоритет 2,все темы,пятидневный,нет


In [9]:
import datetime

def to_dt(year, dot_str):
  return datetime.datetime.strptime(f"{year}-{dot_str}", "%Y-%d.%m")

td = datetime.timedelta

pair_times = [
         (td(hours=9, minutes=0), td(hours=10, minutes=30)),
         (td(hours=10, minutes=40), td(hours=12, minutes=10)),
         (td(hours=12, minutes=50), td(hours=14, minutes=20)),
         (td(hours=14, minutes=30), td(hours=16, minutes=0)),
]

In [10]:
holidays = ['1.1.2020','2.1.2020','3.1.2020','4.1.2020','5.1.2020','6.1.2020','7.1.2020',
            '8.1.2020', '23.2.2020', '8.3.2020','1.5.2020','9.5.2020','12.6.2020','4.11.2020']
holidays = [datetime.datetime.strptime(x, "%d.%m.%Y") for x in holidays]

In [11]:
def make_mat(year):
  # создаем таблицу на год
  dates = pd.date_range(datetime.datetime(year, 1, 1), datetime.datetime(year, 12, 31))
  mat = DataFrame(1, columns=teachers["Преподаватель"], index=dates)
  # убираем выходные дни
  for holiday in holidays:
    mat.loc[holiday, :] = 0
  return mat
mat = make_mat(2020)
mat

Преподаватель,Лебедев М.В.,Красненко А.Г.,Морозов И.О.,Корнеев Р.Л.,Чернов Д.А.,Автеньев Д.Г.,Шабалин К.Н.,Пачко Г.М.,Гладковская А.Ю.,Попова И.О.,Миньков А.С.,Монахов Г.П.,Умняшкин О.В.,Крылова С.А.,Красиков А.В.,Щеглов А.В.,Джавадян А.Э.,Морозов Д.В.,Кузнецов А.А.,Некрасова Л.Д.,Биднюк В.Д.,Блинов А.Б.,Казаков-Прокопьев Т.А.,Селезнев А.В.,Вишняков И.А.,Шарипов А.В.,Митяев В.В.,Патешкин В.Ю.,Каребчик В.А.,Кушниренко П.В.,Четыркин И.А.,Суббота П.В.,Фетисов В.А.,Давыдова Г.В.
2020-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2020-12-28,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2020-12-29,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2020-12-30,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [12]:
def dt_to_day(dt):
  return (dt.date() - datetime.date(dt.year, 1, 1)).days


def read_3(mat, year, filename="data/Приложение №5.xls"):
  holidays = pd.read_excel(filename)
  # находим ряды и столбцы
  holidays = holidays.iloc[6:, 1:]
  holidays.index = holidays.iloc[:, 0]
  holidays.columns = holidays.iloc[0]
  # переименовываем фио
  holidays = holidays.iloc[1:-1, 2:-1]
  holidays.index = holidays.index.map(lambda x: f'{x.split()[0]} {x.split()[1][0]}.{x.split()[2][0]}.' if len(x.split())==3 else x)
  # выделяем декады
  kolvo = holidays.iloc[:, ::2]
  dekada = holidays.iloc[:, 1::2]
  day_start = dekada * 10 + 1
  # добавляем продолжительность отпуска в таблицу
  new_mat = mat.copy()
  for fio, starts in day_start.iterrows():
    for i, start in enumerate(starts):
      try:
        day_start = dt_to_day(datetime.datetime(year, i+1, int(start)))
        day_end = day_start + kolvo.loc[fio].iloc[i]
        new_mat.loc[:, fio].iloc[day_start:day_end + 1] = 0
      except ValueError:
        pass
  
  return new_mat
new_mat = read_3(mat, 2020)
new_mat

Преподаватель,Лебедев М.В.,Красненко А.Г.,Морозов И.О.,Корнеев Р.Л.,Чернов Д.А.,Автеньев Д.Г.,Шабалин К.Н.,Пачко Г.М.,Гладковская А.Ю.,Попова И.О.,Миньков А.С.,Монахов Г.П.,Умняшкин О.В.,Крылова С.А.,Красиков А.В.,Щеглов А.В.,Джавадян А.Э.,Морозов Д.В.,Кузнецов А.А.,Некрасова Л.Д.,Биднюк В.Д.,Блинов А.Б.,Казаков-Прокопьев Т.А.,Селезнев А.В.,Вишняков И.А.,Шарипов А.В.,Митяев В.В.,Патешкин В.Ю.,Каребчик В.А.,Кушниренко П.В.,Четыркин И.А.,Суббота П.В.,Фетисов В.А.,Давыдова Г.В.
2020-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2020-12-28,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2020-12-29,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2020-12-30,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [13]:
!pip install fuzzywuzzy[speedup]

In [14]:
!pip install python-docx 

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
!rm -rf app3
!cp -rf '/content/gdrive/My Drive/Colab Notebooks/PulkovoHack/data/Приложение 3 (docx)' app3
!ls

app3  data  gdrive  out.xlsx  sample_data  task


In [23]:
import os
from docx import Document

def read_3 (fn, t):
  l = list()
  for i, row in enumerate(t.rows[2:]):
    d = dict()
    d['Учебная программа'] = fn[:-4]
    d['Тема'] = row.cells[1].text
    if not d["Тема"] or not ("Тема" in d["Тема"] or "контроль" in d["Тема"].lower()):
      continue
    d['Лекция'] = row.cells[3].text
    d['Практика'] = row.cells[4].text
    l.append(d)
  df = DataFrame(l, columns = ['Учебная программа', 'Тема', 'Лекция', 'Практика'])
  return df

def read_3_all(dn="app3"):
  #находим все файлы с учебными программами 
  directory = dn
  files = os.listdir(directory)
  curriculum = DataFrame(columns=['Учебная программа', 'Тема', 'Лекция', 'Практика'])
  for file in files:
    doc = Document(directory + '/' + file)
    table = doc.tables[0]
    df = read_3(file, table)
    curriculum = curriculum.append(df)
  curriculum = list(curriculum.groupby("Учебная программа"))
  return curriculum

curriculum = read_3_all()
curriculum

[('Базовые компетенции преподавателей Авиационных учебных центров.',
                                      Учебная программа  ... Практика
  0   Базовые компетенции преподавателей Авиационны...  ...     0,25
  1   Базовые компетенции преподавателей Авиационны...  ...      0,5
  2   Базовые компетенции преподавателей Авиационны...  ...     0,25
  3   Базовые компетенции преподавателей Авиационны...  ...     0,25
  4   Базовые компетенции преподавателей Авиационны...  ...     0,25
  5   Базовые компетенции преподавателей Авиационны...  ...        -
  6   Базовые компетенции преподавателей Авиационны...  ...        -
  7   Базовые компетенции преподавателей Авиационны...  ...      0,5
  8   Базовые компетенции преподавателей Авиационны...  ...      0,5
  9   Базовые компетенции преподавателей Авиационны...  ...      0,5
  10  Базовые компетенции преподавателей Авиационны...  ...      0,5
  11  Базовые компетенции преподавателей Авиационны...  ...      0,5
  12  Базовые компет

In [38]:
from fuzzywuzzy import process, fuzz

def gen_pairs(year, calendar, studies, classes, choose_theme=False):
  data = []
  # итерируем по группам
  for i,  (type, aud, start, end, cat, course) in calendar.iterrows():
    start = to_dt(year, start)
    # выбираем учебную программу, которой соответствует категория
    studies_list = [(studies['Учебная программа'].iloc[i], i) for i in studies.index]
    (study, index), _ = process.extractOne(f"{cat} {course}", studies_list, scorer=fuzz.token_set_ratio)
    # выбираем программу, соответствующую выбранной
    if choose_theme:
      (plan_name, study_plan), _ = process.extractOne(f"{cat} {course}", curriculum, scorer=fuzz.token_set_ratio)
    study = studies.iloc[index]
    # выбираем учителей, преподающих категорию
    available_teachers = []
    for i, programs in teachers["Учебные программы"].astype(str).iteritems():
      programs = [int(x) - 1 for x in programs.split(';')]
      if index in programs:
        available_teachers.append(teachers["Преподаватель"].iloc[i])
    # готовимся к циклу, где мы распишем занятия по парам
    hours = int(study["Занятия в классе(теоретические и практические занятия),  ак. часов"])
    total_hours, hours_left = hours, hours
    day = start
    pair = 0
    # расписываем все часы
    while hours_left > 0:
      # если мы прошли все пары за день, переходим на следующий день
      if pair == len(pair_times):
        pair = 0
        day += td(days=1)
        continue
      # если выходной, пропускаем
      if day.weekday() > 5:
        day += td(days=1)
        continue
      # смотрим, кто в отпуске
      available = []
      for teacher in available_teachers:
        if new_mat[teacher].iloc[dt_to_day(day)] == 1:
          available.append(teacher)
      # пропускаем если нет учителей
      if not available:
        day += td(days=1)
        continue
      # выбираем подходящие аудитории
      auds = []
      for i, cls in classes.iterrows():
        auds.append(cls["Аудитория"])
      # смотрим, когда пара идет
      pair_start, pair_end = pair_times[pair]
      hour_start = total_hours - hours_left
      hour_end = hour_start + 2
      # выбираем тему (медленно)
      if choose_theme:
        plan_hours = 0
        theme = []
        for _, group in study_plan.iterrows():
          t = group["Тема"]
          try:
            l = float(group["Лекция"].strip())
          except ValueError:
            l = 0
          try:
            p = float(group["Практика"])
          except ValueError:
            p = 0
          new_plan_hours = plan_hours + l + p
          if (hour_start < plan_hours < hour_end) or (hour_start < new_plan_hours < hour_end):
            theme.append(t)
          plan_hours = new_plan_hours
        theme = '\n'.join(theme)
      else:
        theme = ""
      # добавляем пару
      data.append((day + pair_start, day + pair_end, hour_start, hour_end, auds, cat, type, course, available, index, theme))
      # переходим на следующую
      hours_left -= 2 # два академических часа
      pair += 1
  df = DataFrame(data, columns=["start", "end", "aca_start", "aca_end", "aud", "cat", "type", "course", "teachers", "study", "theme"])
  return df

pairs_ = gen_pairs(2020, example_calendar, studies, classes)
pairs_

,start,end,aca_start,aca_end,aud,cat,type,course,teachers,study,theme
0,2020-01-13 09:00:00,2020-01-13 10:30:00,0,2,"[105, 106, 107, 414б, 414б-1, 416а,б, 422, 309...",Обучение авиационного персонала,Программа повышения квалификации,Досмотр,"[Миньков А.С., Монахов Г.П., Умняшкин О.В., Ще...",29,
1,2020-01-13 10:40:00,2020-01-13 12:10:00,2,4,"[105, 106, 107, 414б, 414б-1, 416а,б, 422, 309...",Обучение авиационного персонала,Программа повышения квалификации,Досмотр,"[Миньков А.С., Монахов Г.П., Умняшкин О.В., Ще...",29,
2,2020-01-13 12:50:00,2020-01-13 14:20:00,4,6,"[105, 106, 107, 414б, 414б-1, 416а,б, 422, 309...",Обучение авиационного персонала,Программа повышения квалификации,Досмотр,"[Миньков А.С., Монахов Г.П., Умняшкин О.В., Ще...",29,
3,2020-01-13 14:30:00,2020-01-13 16:00:00,6,8,"[105, 106, 107, 414б, 414б-1, 416а,б, 422, 309...",Обучение авиационного персонала,Программа повышения квалификации,Досмотр,"[Миньков А.С., Монахов Г.П., Умняшкин О.В., Ще...",29,
4,2020-01-14 09:00:00,2020-01-14 10:30:00,8,10,"[105, 106, 107, 414б, 414б-1, 416а,б, 422, 309...",Обучение авиационного персонала,Программа повышения квалификации,Досмотр,"[Миньков А.С., Монахов Г.П., Умняшкин О.В., Ще...",29,
...,...,...,...,...,...,...,...,...,...,...,...
1627,2020-12-22 14:30:00,2020-12-22 16:00:00,14,16,"[105, 106, 107, 414б, 414б-1, 416а,б, 422, 309...",Обучение неавиационного персонала,,Безопасность полетов,"[Биднюк В.Д., Блинов А.Б., Казаков-Прокопьев Т...",39,
1628,2020-12-23 09:00:00,2020-12-23 10:30:00,16,18,"[105, 106, 107, 414б, 414б-1, 416а,б, 422, 309...",Обучение неавиационного персонала,,Безопасность полетов,"[Биднюк В.Д., Блинов А.Б., Казаков-Прокопьев Т...",39,
1629,2020-12-23 10:40:00,2020-12-23 12:10:00,18,20,"[105, 106, 107, 414б, 414б-1, 416а,б, 422, 309...",Обучение неавиационного персонала,,Безопасность полетов,"[Биднюк В.Д., Блинов А.Б., Казаков-Прокопьев Т...",39,
1630,2020-12-23 12:50:00,2020-12-23 14:20:00,20,22,"[105, 106, 107, 414б, 414б-1, 416а,б, 422, 309...",Обучение неавиационного персонала,,Безопасность полетов,"[Биднюк В.Д., Блинов А.Б., Казаков-Прокопьев Т...",39,


In [39]:
from scipy.optimize import linear_sum_assignment

def divide_classes(pairs):
  new_pairs = pairs.copy()
  # распределять надо среди параллельных пар
  for period, group in pairs.groupby(["start", "end"]):
    # распределяем по аудиториям
    # clss = set(group["aud"].apply(pd.Series).stack().reset_index(drop=True))
    # составляем матрицу совместимости кабинетов и преподов
    weights = np.zeros((len(group), len(classes)))
    rows = np.zeros(len(group))
    for i, (k, grp) in enumerate(group.iterrows()):
      for j, cls in classes.iterrows():
        # проверяем, возможно ли вообще их совместить
        impossible = False
        
        # проверяем по предметам
        study = studies.iloc[grp.study]
        if cls["Без парт"] and not study["Без парт"]:
          impossible = True
        if study["Доска"] and not cls["Доска"]:
          impossible = True

        # проверяем по дисциплинам
        disciplines = cls["Подходит для дисциплин"].lower()
        if "все" not in disciplines:
          check_is = True
          if "кроме" in disciplines:
            check_is = False
          if (grp["cat"].lower() in disciplines) != check_is:
            impossible = True
        
        if impossible:
          weight = -100
        else:
          # вычисляем вес
          weight = 0
          # проверяем по преимуществам
          advantage = cls["Преимущество у дисциплины"]
          if grp["cat"] in advantage or grp["course"] in advantage:
            weight += 100
          #.
          if "422" in str(cls["Аудитория"]):
            weight -= 50
        #
        weights[i, j] = weight
        rows[i] = k
    # используем венгерский алгоритм для поиска соответствий
    _, cols = linear_sum_assignment(weights, maximize=True)
    # задаем выбранные аудитории
    for col, row in zip(cols, rows):
      new_pairs.loc[row, "aud"] = classes.loc[col, "Аудитория"]
  #
  return new_pairs
    
pairs_class = divide_classes(pairs_)
pairs_class

,start,end,aca_start,aca_end,aud,cat,type,course,teachers,study,theme
0,2020-01-13 09:00:00,2020-01-13 10:30:00,0,2,414б,Обучение авиационного персонала,Программа повышения квалификации,Досмотр,"[Миньков А.С., Монахов Г.П., Умняшкин О.В., Ще...",29,
1,2020-01-13 10:40:00,2020-01-13 12:10:00,2,4,414б,Обучение авиационного персонала,Программа повышения квалификации,Досмотр,"[Миньков А.С., Монахов Г.П., Умняшкин О.В., Ще...",29,
2,2020-01-13 12:50:00,2020-01-13 14:20:00,4,6,414б,Обучение авиационного персонала,Программа повышения квалификации,Досмотр,"[Миньков А.С., Монахов Г.П., Умняшкин О.В., Ще...",29,
3,2020-01-13 14:30:00,2020-01-13 16:00:00,6,8,414б,Обучение авиационного персонала,Программа повышения квалификации,Досмотр,"[Миньков А.С., Монахов Г.П., Умняшкин О.В., Ще...",29,
4,2020-01-14 09:00:00,2020-01-14 10:30:00,8,10,414б,Обучение авиационного персонала,Программа повышения квалификации,Досмотр,"[Миньков А.С., Монахов Г.П., Умняшкин О.В., Ще...",29,
...,...,...,...,...,...,...,...,...,...,...,...
1627,2020-12-22 14:30:00,2020-12-22 16:00:00,14,16,414б,Обучение неавиационного персонала,,Безопасность полетов,"[Биднюк В.Д., Блинов А.Б., Казаков-Прокопьев Т...",39,
1628,2020-12-23 09:00:00,2020-12-23 10:30:00,16,18,107,Обучение неавиационного персонала,,Безопасность полетов,"[Биднюк В.Д., Блинов А.Б., Казаков-Прокопьев Т...",39,
1629,2020-12-23 10:40:00,2020-12-23 12:10:00,18,20,107,Обучение неавиационного персонала,,Безопасность полетов,"[Биднюк В.Д., Блинов А.Б., Казаков-Прокопьев Т...",39,
1630,2020-12-23 12:50:00,2020-12-23 14:20:00,20,22,107,Обучение неавиационного персонала,,Безопасность полетов,"[Биднюк В.Д., Блинов А.Б., Казаков-Прокопьев Т...",39,


In [40]:
from scipy.optimize import linear_sum_assignment

def divide_teachers(pairs):
  new_pairs = pairs.copy()
  # распределять надо среди параллельных пар
  for period, group in pairs.groupby(["start", "end"]):
    # распределяем учителей
    available = group["teachers"]
    all_available = list(set(sum(list(available), [])))
    # составляем матрицу совместимости кабинетов и преподов
    weights = np.zeros((len(group), len(all_available)))
    rows = np.zeros(len(group))
    for i, (k, grp) in enumerate(group.iterrows()):
      for j, cls in enumerate(all_available):
        if cls in grp["teachers"]:
          cls = teachers[teachers['Преподаватель'] == cls].iloc[0]
          # вычисляем вес
          weight = 10000
          # проверяем по дисциплинам
          if grp["course"] in cls["Дисциплина"]:
            weight += 100
          # используем приоритет
          try:
            priority = int(cls["Приоритет при распределении"])
            weight += 10000 / priority
          except ValueError:
            pass
        else:
          weight = 0
        weights[i, j] = weight
        rows[i] = k
    # используем венгерский алгоритм для поиска соответствий
    _, cols = linear_sum_assignment(weights, maximize=True)
    # задаем выбранные аудитории
    for col, row in zip(cols, rows):
      new_pairs.loc[row, "teachers"] = all_available[col]
  #
  return new_pairs

pairs = divide_teachers(pairs_class)
pairs

,start,end,aca_start,aca_end,aud,cat,type,course,teachers,study,theme
0,2020-01-13 09:00:00,2020-01-13 10:30:00,0,2,414б,Обучение авиационного персонала,Программа повышения квалификации,Досмотр,Умняшкин О.В.,29,
1,2020-01-13 10:40:00,2020-01-13 12:10:00,2,4,414б,Обучение авиационного персонала,Программа повышения квалификации,Досмотр,Умняшкин О.В.,29,
2,2020-01-13 12:50:00,2020-01-13 14:20:00,4,6,414б,Обучение авиационного персонала,Программа повышения квалификации,Досмотр,Умняшкин О.В.,29,
3,2020-01-13 14:30:00,2020-01-13 16:00:00,6,8,414б,Обучение авиационного персонала,Программа повышения квалификации,Досмотр,Умняшкин О.В.,29,
4,2020-01-14 09:00:00,2020-01-14 10:30:00,8,10,414б,Обучение авиационного персонала,Программа повышения квалификации,Досмотр,Умняшкин О.В.,29,
...,...,...,...,...,...,...,...,...,...,...,...
1627,2020-12-22 14:30:00,2020-12-22 16:00:00,14,16,414б,Обучение неавиационного персонала,,Безопасность полетов,Селезнев А.В.,39,
1628,2020-12-23 09:00:00,2020-12-23 10:30:00,16,18,107,Обучение неавиационного персонала,,Безопасность полетов,Биднюк В.Д.,39,
1629,2020-12-23 10:40:00,2020-12-23 12:10:00,18,20,107,Обучение неавиационного персонала,,Безопасность полетов,Биднюк В.Д.,39,
1630,2020-12-23 12:50:00,2020-12-23 14:20:00,20,22,107,Обучение неавиационного персонала,,Безопасность полетов,Биднюк В.Д.,39,


In [41]:
from collections import Counter

def duplicates(lst):
  return [k for k,v in Counter(lst).items() if v > 1]

def err_check(pairs):
  for period, group in pairs.groupby(["start", "end"]):
    auds = list(group.aud)
    for dup in duplicates(auds):
      yield f"Накладка с аудиторией {dup} с {period[0]} до {period[1]}"
    teachers = list(group.teachers)
    for dup in duplicates(teachers):
      yield f"Накладка с учителем {dup} с {period[0]} до {period[1]}"
for err in err_check(pairs):
  print(err)

In [42]:
def metrics(calendar, pairs):
  months = []
  for month in range(1, 12+1):
    pair = pairs[pairs["start"].apply(lambda x: x.month == month)]
    clss = {}
    for _, cls in classes.iterrows():
      cls = cls["Аудитория"]
      clss[cls] = len(pair[pair["aud"] == cls]) / (4 * 28)
    events = {}
    for name, group in calendar.groupby("name"):
      # if type != '':
      #   name = f"{name}: {type}"
      events[name] = len(group[group["start"].apply(lambda x: int(x.split('.')[1]) == month)])
    hours = {}
    for t, teacher in teachers.iterrows():
      teacher = teacher["Преподаватель"]
      hours[teacher] = len(pair[pair["teachers"] == teacher]) * 2
    months.append((clss, events, hours))
  final_month = ({}, {}, {})
  for types in months:
    for i, objects in enumerate(types):
      for name, count in objects.items():
        try:
          final_month[i][name] += count
        except KeyError:
          final_month[i][name] = count
  months.append(final_month)
  return months

def print_metric(metric):
  for i, (clss, events, teas) in enumerate(metric):
    if i < 12:
      print(f"Месяц {i + 1}:")
    else:
      print("Итого:")
    print(" Проведено групп:")
    for name, count in events.items():
      if count == 0:
        continue
      print(f"  {name} - {count}")
    print(" Использование кабинетов:")
    for cls, pct in clss.items():
      if i == 12:
        pct /= 12
      if pct < 0.01:
        continue
      print(f"  Кабинет {cls}: {int(pct * 100)}%")
    print(" Загрузка преподавателей:")
    for name, hours in teas.items():
      if hours == 0:
        continue
      print(f"  {name}: {hours}ч")

metric = metrics(example_calendar, pairs)
print_metric(metric)

Месяц 1:
 Проведено групп:
  Досмотр - 4
  Опасные грузы. 10 категория - 2
  Опасные грузы. 8 категория - 1
  Опасные грузы. 9 категория - 4
  Охрана аэропорта - 1
  Пассажирские перевозки - 1
  Перронный контроль - 2
 Использование кабинетов:
  Кабинет 105: 7%
  Кабинет 106: 25%
  Кабинет 107: 35%
  Кабинет 414б: 35%
 Загрузка преподавателей:
  Автеньев Д.Г.: 32ч
  Попова И.О.: 40ч
  Умняшкин О.В.: 72ч
  Красиков А.В.: 32ч
  Щеглов А.В.: 40ч
  Давыдова Г.В.: 16ч
Месяц 2:
 Проведено групп:
  DCS Астра - 1
  Безопасность полетов - 1
  Досмотр - 4
  Опасные грузы. 8 категория - 1
  Опасные грузы. 9 категория - 4
  Охрана аэропорта - 1
  Пассажирские перевозки - 2
  Перронный контроль - 2
  Центровка и контроль загрузки ВС - 2
 Использование кабинетов:
  Кабинет 105: 14%
  Кабинет 106: 32%
  Кабинет 107: 42%
  Кабинет 414б: 75%
  Кабинет 422: 10%
 Загрузка преподавателей:
  Красненко А.Г.: 128ч
  Гладковская А.Ю.: 8ч
  Попова И.О.: 32ч
  Умняшкин О.В.: 72ч
  Красиков А.В.: 32ч
  Щеглов А.

In [43]:
from openpyxl.utils import get_column_letter

writer = pd.ExcelWriter('out.xlsx')
for week, group in pairs.groupby(pairs.start.dt.week):
  week_name = str(week)
  group = group.astype(str)
  group.to_excel(writer, index=False, sheet_name=week_name)
  for i, column in enumerate(group.columns):
    length = group[column].str.len().max()
    writer.sheets[week_name].column_dimensions[get_column_letter(i + 1)].width = length + 5
writer.save()